In [1]:
!pip install --upgrade --force-reinstall git+https://github.com/PatoDckY/cec_problems.git
import numpy as np
import pandas as pd
import time
import inspect
from openpyxl.styles import PatternFill

from cec_problems import ProblemaC01
from cec_problems import ProblemaC02
from cec_problems import ProblemaC03
from cec_problems import ProblemaC04
from cec_problems import ProblemaC05
from cec_problems import ProblemaC06
from cec_problems import ProblemaC07
from cec_problems import ProblemaC08
from cec_problems import ProblemaC09
from cec_problems import ProblemaC10
from cec_problems import ProblemaC11
from cec_problems import ProblemaC12
from cec_problems import ProblemaC13
from cec_problems import ProblemaC14
from cec_problems import ProblemaC15
from cec_problems import ProblemaC16
from cec_problems import ProblemaC17
from cec_problems import ProblemaC18

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/PatoDckY/cec_problems.git to c:\users\kangu\appdata\local\temp\pip-req-build-jcz216cb
  Resolved https://github.com/PatoDckY/cec_problems.git to commit c1d52c64776f8a2a36532948bb656064f22dbce5
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.3.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.2-cp313-cp313-win_amd64.whl (12.8 MB)
  Created wheel for cec_problems: filename=cec_problems-0.1.0-py3-none-any.whl size=4565 sha256=238d5a63f934cf7bf9cace4468f1591130eb1e9fc032fc1a67711893e0e3b80d
  Stored in directory: C:\Users\kangu\AppData\Local\Temp\pip-ephem-wheel-cache

  Running command git clone --filter=blob:none --quiet https://github.com/PatoDckY/cec_problems.git 'C:\Users\kangu\AppData\Local\Temp\pip-req-build-jcz216cb'
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Función para generar un individuo aleatorio con distribución normal
def generar_individuo_normal(lower, upper):
    media = (lower + upper) / 2
    sigma = (upper - lower) / 6  # Aproximadamente 99.7% dentro de límites
    individuo = np.random.normal(loc=media, scale=sigma)
    return individuo

# Función para generar una población
def generar_poblacion(tamano_poblacion, lower_bounds, upper_bounds):
    return np.array([generar_individuo_normal(lower_bounds, upper_bounds) for _ in range(tamano_poblacion)])


In [3]:
def corregir_individuo(individuo, lower_bounds, upper_bounds, funcion_limite):
  if(not es_valido(individuo, lower_bounds, upper_bounds)):
    return funcion_limite(individuo, lower_bounds, upper_bounds)
  else:
    return individuo



def validar_poblacion(poblacion, lower_bounds, upper_bounds, funcion_limite,
                      poblacion_actual=None, x_best=None, evaluate_fn=None,
                      current_generation=None, total_generations=None):

    poblacion_corregida = []

    # Obtener la firma de la función
    firma = inspect.signature(funcion_limite)
    parametros = list(firma.parameters.keys())

    for individuo in poblacion:
        if not es_valido(individuo, lower_bounds, upper_bounds):
            # Llamar solo con los argumentos que la función acepte
            if len(parametros) == 3:
                individuo_corregido = funcion_limite(individuo, lower_bounds, upper_bounds)
            else:
                individuo_corregido = funcion_limite(
                    individuo, lower_bounds, upper_bounds,
                    poblacion=poblacion_actual,
                    x_best=x_best,
                    evaluate_fn=evaluate_fn,
                    current_generation=current_generation,
                    total_generations=total_generations
                )
        else:
            individuo_corregido = individuo
        poblacion_corregida.append(individuo_corregido)

    return np.array(poblacion_corregida)


def es_valido(individuo, lower_bounds, upper_bounds):
    return np.all((individuo >= lower_bounds) & (individuo <= upper_bounds))

In [4]:
def metodo_mut(_, Xb, Xc, F, X_best, x_target=None):
    return X_best + (F * (Xb - Xc))
    
def mutar_poblacion(poblacion, F, funcion_mutacion, funcion_aptitud):
    N = len(poblacion)
    mutantes = []

    # Calcular el mejor individuo (para estrategias que lo requieran)
    fitness_array, _ = evaluar_poblacion(poblacion, funcion_aptitud)
    idx_best = np.argmin(fitness_array)
    X_best = poblacion[idx_best]

    for i in range(N):
        x_target = poblacion[i]
        indices = [j for j in range(N) if j != i]
        seleccionados = np.random.choice(indices, size=3, replace=False)
        Xa, Xb, Xc = poblacion[seleccionados[0]], poblacion[seleccionados[1]], poblacion[seleccionados[2]]

        # Llamar a la función de mutación con soporte para el mejor individuo
        Vi = funcion_mutacion(Xa, Xb, Xc, F, X_best,x_target)
        mutantes.append(Vi)

    return np.array(mutantes)


In [5]:
def cruza_binomial(poblacion_original, poblacion_mutada, tasa_cruza):
    N, D = poblacion_original.shape #(N) numero de individuos, (D) Dimension, Tamaño de la Matriz
    hijos = np.empty_like(poblacion_original)

    for i in range(N):
        j_rand = np.random.randint(D)  # Asegura al menos una componente de Vi
        for j in range(D):
            if np.random.rand() < tasa_cruza or j == j_rand:
                hijos[i, j] = poblacion_mutada[i, j]
            else:
                hijos[i, j] = poblacion_original[i, j]

    return hijos

In [6]:
def seleccionar_mejores(poblacion_original, poblacion_prueba, fitness_original, fitness_prueba):
    nueva_poblacion = []

    for ind_orig, ind_prueba, fit_orig, fit_prueba in zip(poblacion_original, poblacion_prueba, fitness_original, fitness_prueba):
        if fit_prueba <= fit_orig:
            nueva_poblacion.append(ind_prueba)
        else:
            nueva_poblacion.append(ind_orig)

    return np.array(nueva_poblacion)

def seleccionar_mejores_deb(poblacion_original, poblacion_prueba,fitness_orig, viol_orig,fitness_prueba, viol_prueba):

    nueva_poblacion = []
    fitness_sel     = []
    viol_sel        = []

    for ind1, ind2, f1, v1, f2, v2 in zip(poblacion_original,poblacion_prueba,fitness_orig,viol_orig,fitness_prueba,viol_prueba):
        ganador, f_gan, v_gan = deb_selection(ind1, f1, v1, ind2, f2, v2)
        nueva_poblacion.append(ganador)
        fitness_sel.append(f_gan)
        viol_sel.append(v_gan)

    return (np.array(nueva_poblacion),np.array(fitness_sel),np.array(viol_sel))


In [7]:
# Función que encuentra el mejor individuo y su fitness
def obtener_mejor_individuo(poblacion, fitness):
    idx_mejor = np.argmin(fitness)  # índice del menor valor de fitness
    return poblacion[idx_mejor], fitness[idx_mejor]

def obtener_mejor_deb(poblacion, fitness_array, viol_array):

    # Inicializamos con el primero
    ind_best = poblacion[0]
    f_best   = fitness_array[0]
    v_best   = viol_array[0]

    # Recorremos el resto
    for ind, f, v in zip(poblacion[1:], fitness_array[1:], viol_array[1:]):

        ind_best, f_best,v_best=deb_selection(ind_best, f_best, v_best, ind, f, v)

    return ind_best, f_best, v_best



In [8]:
def evaluar_poblacion(poblacion, evaluate_fn):
    # Llamamos a evaluate() sobre cada individuo y separamos resultados
    resultados = np.array([evaluate_fn(ind) for ind in poblacion])
    fitness_array    = resultados[:, 0]
    violaciones_array = resultados[:, 1]
    return fitness_array, violaciones_array

def deb_selection(ind1, f1, v1, ind2, f2, v2):
    feas1 = (v1 <= 0)
    feas2 = (v2 <= 0)

    # Ambos factibles → mejor fitness
    if feas1 and feas2:
        if f1 <= f2:
            return ind1, f1, v1
        else:
            return ind2, f2, v2

    # Uno factible → lo conservamos
    elif feas1:
        return ind1, f1, v1
    elif feas2:
        return ind2, f2, v2

    # Ambos infactibles → menor violación
    elif v1 <= v2:
        return ind1, f1, v1
    else:
        return ind2, f2, v2

In [9]:
#propuesta metodo

def Limcor(vector_inviable, limites_inferiores, limites_superiores, x_best=None, poblacion=None,
           evaluate_fn=None, current_generation=None, total_generations=None):
    """
    Parámetros:
    - vector_inviable (np.array): El vector (individuo) que necesita corrección.
    - limites_inferiores (np.array): Límites inferiores del espacio de búsqueda.
    - limites_superiores (np.array): Límites superiores del espacio de búsqueda.
    - x_best (np.array, opcional): El mejor individuo de la población hasta el momento.
    - poblacion (list, opcional): La población actual de individuos.
    - evaluate_fn (function, opcional): Función para evaluar un individuo y su violación.
    - current_generation (int, opcional): Generación actual del algoritmo.
    - total_generations (int, opcional): Número total de generaciones.

    Retorna:
    El vector corregido, garantizado para estar dentro de los límites.
    """
    
    # 1. Inicialización
    limcor_vector = np.copy(vector_inviable)
    num_variables = len(vector_inviable)
    porcentaje_vecinos = 0.2

    # 2. Recolectar vecinos factibles
    # Se obtienen los individuos de la población que están dentro de los límites
    # y no violan las restricciones (si evaluate_fn está disponible).
    vecinos_factibles = []
    if poblacion is not None and evaluate_fn is not None:
        for vecino in poblacion:
            # es_valido es una función auxiliar que verifica los límites
            if es_valido(vecino, limites_inferiores, limites_superiores):
                # evaluate_fn devuelve la aptitud y la violación (si aplica)
                _, violacion = evaluate_fn(vecino)
                if violacion <= 0:
                    vecinos_factibles.append(vecino)
    vecinos_factibles = np.array(vecinos_factibles)

    # 3. Iterar sobre cada variable del vector inviable
    for j in range(num_variables):
        x_j = limcor_vector[j]
        li, ui = limites_inferiores[j], limites_superiores[j]

        # 4. Verificar si la variable está fuera de los límites
        if x_j < li or x_j > ui:
            
            # Caso especial: Si el límite inferior y superior son iguales, se asigna ese valor.
            if li == ui:
                limcor_vector[j] = li
                continue

            # 5. Calcular distancia relativa y peso gamma
            rango = ui - li
            if x_j < li:
                D_rel = (li - x_j) / rango
                limite_violado = li
            else:
                D_rel = (x_j - ui) / rango
                limite_violado = ui
            
            # Se asegura que la distancia relativa esté entre 0 y 1.
            D_rel = np.clip(D_rel, 0, 1)
            # Gamma (peso para la explotación) es inversamente proporcional a la distancia.
            # Cuanto más cerca del límite, mayor es el peso de la explotación.
            gamma = max(0.1, 1 - D_rel)

            # 6. Determinar el punto de referencia para la exploración (m_j)
            m_j = None
            if len(vecinos_factibles) > 0:
                # Si hay vecinos factibles, se toma la media de un subconjunto aleatorio.
                M = max(1, round(len(vecinos_factibles) * porcentaje_vecinos))
                indices = np.random.choice(len(vecinos_factibles), min(M, len(vecinos_factibles)), replace=False)
                m_j = np.mean(vecinos_factibles[indices, j])
            elif x_best is not None and j < len(x_best):
                # Si no hay vecinos factibles, se usa la variable del mejor individuo.
                m_j = x_best[j]
            else:
                # Si no hay nada, se elige un valor aleatorio dentro de los límites.
                m_j = np.random.uniform(li, ui)

            # 7. Calcular el peso lambda para el balance exploración-explotación
            lambda_ = 0.0
            if total_generations:
                # Lambda aumenta con el progreso de la generación, favoreciendo la exploración
                # en las primeras etapas y la explotación en las últimas.
                progreso = (current_generation or 0) / total_generations
                lambda_ = np.tanh(D_rel) * progreso

            # 8. Calcular el nuevo valor de la variable
            ref_explotacion = x_best[j] if x_best is not None and j < len(x_best) else limite_violado

            p_explotacion = gamma * ref_explotacion + (1 - gamma) * limite_violado
            
            # La nueva variable es un promedio ponderado entre la explotación y la exploración.
            limcor_vector[j] = (1 - lambda_) * p_explotacion + lambda_ * m_j

    # 9. Asegurar que el vector corregido esté dentro de los límites finales
    corregido = np.clip(limcor_vector, limites_inferiores, limites_superiores)
    return corregido

In [10]:

#metodos 

def reflection(individuo, lower_bounds, upper_bounds):
    individuo = np.array(individuo)
    lower_bounds = np.array(lower_bounds)
    upper_bounds = np.array(upper_bounds)

    r = upper_bounds - lower_bounds
    valorNorm = (individuo - lower_bounds)
    s = np.floor(valorNorm / r)
    posicion_en_segmento = valorNorm - s * r
    individuo_reflejado = np.zeros_like(individuo)
    cond_par = (s % 2 == 0)
    individuo_reflejado[cond_par] = lower_bounds[cond_par] + posicion_en_segmento[cond_par]
    individuo_reflejado[~cond_par] = upper_bounds[~cond_par] - posicion_en_segmento[~cond_par]

    return individuo_reflejado

def wrapping(individuo, lower_bounds, upper_bounds):
    corregido = individuo.copy()
    for j in range(len(individuo)):
        lj = lower_bounds[j]
        uj = upper_bounds[j]
        rho = uj - lj

        if individuo[j] < lj:
            corregido[j] = uj - ((lj - individuo[j]) % rho)
        elif individuo[j] > uj:
            corregido[j] = lj + ((individuo[j] - uj) % rho)
        # si está dentro del rango, no se hace nada

    return corregido

def boundary(individuo, lower_bounds, upper_bounds):
    corregido = np.clip(individuo, lower_bounds, upper_bounds)
    return corregido

def random_reset(individuo, lower_bounds, upper_bounds):
    corregido = individuo.copy()
    for i in range(len(individuo)):
        if individuo[i] < lower_bounds[i] or individuo[i] > upper_bounds[i]:
            corregido[i] = np.random.uniform(lower_bounds[i], upper_bounds[i])
    return corregido

def evolutionary(x, lb, ub, x_best):
    x_new = x.copy()
    for j in range(len(x)):
        if x[j] < lb[j]:
            r = np.random.uniform()
            x_new[j] = r * lb[j] + (1 - r) * x_best[j]
        elif x[j] > ub[j]:
            r = np.random.uniform()
            x_new[j] = r * ub[j] + (1 - r) * x_best[j]
    return x_new



In [11]:
def wrapper_evolutionary(individuo, lower_bounds, upper_bounds, **kwargs):
    # Extraemos solo el x_best de los kwargs para pasárselo a la función original
    x_best_val = kwargs.get('x_best')
    if x_best_val is None:
        # Esto es un caso de seguridad. Si por alguna razón x_best no llega,
        # podríamos aplicar un clipping o lanzar un error, dependiendo de tu lógica.
        print("Advertencia: x_best no proporcionado a wrapper_evolutionary. Aplicando clipping como fallback.")
        return np.clip(individuo, lower_bounds, upper_bounds)
    
    # Llamamos a tu función 'evolutionary' original con la firma exacta que quieres.
    return evolutionary(individuo, lower_bounds, upper_bounds, x_best_val)

In [12]:
def ejecutar_ed(tamano_poblacion, lower_bounds, upper_bounds, evaluate_fn, F, CR, func_limite, generaciones, func_mutacion):
    # 1) Población inicial
    poblacion = generar_poblacion(tamano_poblacion, lower_bounds, upper_bounds)
    fitness_p, violc_p = evaluar_poblacion(poblacion, evaluate_fn)
    
    # Obtener el mejor individuo inicial para el wrapper
    ind_best_gen_start, f_best_gen_start, v_best_gen_start = obtener_mejor_deb(poblacion, fitness_p, violc_p)

    mejores_por_generacion = []
    
    # Este x_best_for_aedr_and_evolutionary se actualiza en cada generación
    x_best_for_aedr_and_evolutionary = ind_best_gen_start


    for gen in range(1, generaciones + 1):
        # 2) Mutación (se pasa evaluate_fn como función de aptitud)
        pobl_mutada = mutar_poblacion(poblacion, F, func_mutacion, evaluate_fn)

        # 3) Cruza
        pobl_prueba = cruza_binomial(poblacion, pobl_mutada, CR)

        # 4) Aplicar límites
        # Se pasa x_best_for_aedr_and_evolutionary a validar_poblacion, y esta a su vez
        # lo pasará como 'x_best' a la func_limite
        pobl_prueba_validada = validar_poblacion(
            pobl_prueba, lower_bounds, upper_bounds, func_limite, # Aquí se usa la función de límite original
            poblacion_actual=poblacion, x_best=x_best_for_aedr_and_evolutionary, evaluate_fn=evaluate_fn,
            current_generation=gen, total_generations=generaciones
        )

        # 5) Evaluar la población prueba
        fitness_p_v, violc_p_v = evaluar_poblacion(pobl_prueba_validada, evaluate_fn)

        # 6) Selección DEB
        poblacion_seleccionada, fit_selec, viol_selec = seleccionar_mejores_deb(poblacion, pobl_prueba_validada,fitness_p, violc_p,fitness_p_v, violc_p_v)

        # 7) Obtener mejor individuo
        ind_best, f_best, v_best = obtener_mejor_deb(poblacion_seleccionada, fit_selec, viol_selec)
        mejores_por_generacion.append({'individuo': ind_best.copy(),'fitness': f_best,'violation': v_best,'factible': v_best <= 0})

        # 8) Preparar siguiente generación
        poblacion = poblacion_seleccionada
        fitness_p = fit_selec
        violc_p = viol_selec

        # Actualizar el x_best para las funciones de límite que lo usan en la siguiente generación
        x_best_for_aedr_and_evolutionary = ind_best
        
    return mejores_por_generacion


In [13]:
def ejecutar_combinaciones_ED_limites(
    clases_problema,
    nombres_problemas,
    metodos_limite,
    nombres_limite,
    num_corridas,
    tamano_poblacion,
    offset,
    F,
    CR,
    metodo_mut,
    generaciones
):
    for cls_prob, nombre_prob in zip(clases_problema, nombres_problemas):
        # --- Instanciamos el problema con offset (o fallback) ---
        try:
            prob = cls_prob(offset)
        except TypeError:
            prob = cls_prob()
            if hasattr(prob, 'set_offset'):
                prob.set_offset(offset)
            elif hasattr(prob, 'offset'):
                prob.offset = np.asarray(offset, dtype=float)

        # --- Obtenemos límites ---
        if hasattr(prob, 'get_limites'):
            lower_bounds, upper_bounds = prob.get_limites()
        elif hasattr(prob, 'limites'):
            try:
                lower_bounds, upper_bounds = prob.limites()
            except TypeError:
                lower_bounds, upper_bounds = prob.limites(offset)
        else:
            lower_bounds = prob.lower_bounds
            upper_bounds = prob.upper_bounds

        evaluate_fn = prob.evaluate

        archivo = f"ED_Limites_{nombre_prob}.xlsx"
        with pd.ExcelWriter(archivo, engine="openpyxl") as writer:
            for metodo_lim, nombre_lim in zip(metodos_limite, nombres_limite):
                print(f"\n🔁 Problema = {nombre_prob} | Límite = {nombre_lim}")

                # --- Acumuladores ---
                matriz = np.zeros((generaciones, num_corridas))
                tiempos = []
                factibles_final = []
                violaciones_final = []

                # --- Corridas ---
                for j in range(num_corridas):
                    inicio = time.time()
                    
                    # Población inicial para cada corrida
                    poblacion = generar_poblacion(tamano_poblacion, lower_bounds, upper_bounds)
                    fitness_p, violc_p = evaluar_poblacion(poblacion, evaluate_fn)
                    
                    # Obtener el mejor individuo inicial para esta corrida
                    ind_best_corrida, f_best_corrida, v_best_corrida = obtener_mejor_deb(poblacion, fitness_p, violc_p)

                    # Determinar la función de límite a usar en esta corrida
                    # Si es "Evolutionary", se usa el wrapper. Si es otro, se usa directamente.
                    func_limite_a_usar = None
                    if nombre_lim == "Evolutionary":
                        func_limite_a_usar = wrapper_evolutionary
                    else:
                        func_limite_a_usar = metodo_lim

                    resultados = []
                    
                    # El mejor individuo de la generación actual, se actualizará en cada iteración
                    x_best_for_current_gen = ind_best_corrida
                    
                    for gen in range(1, generaciones + 1):
                        # Mutación
                        pobl_mutada = mutar_poblacion(poblacion, F, metodo_mut, evaluate_fn)
                        
                        # Cruza
                        pobl_prueba = cruza_binomial(poblacion, pobl_mutada, CR)
                        
                        # Aplicar límites (esto es lo que estamos evaluando)
                        # Pasar todos los kwargs necesarios a validar_poblacion,
                        # que a su vez los pasa a func_limite_a_usar
                        pobl_prueba_validada = validar_poblacion(
                            pobl_prueba,
                            lower_bounds,
                            upper_bounds,
                            func_limite_a_usar, 
                            poblacion_actual=poblacion,      
                            x_best=x_best_for_current_gen,   
                            evaluate_fn=evaluate_fn,         
                            current_generation=gen,         
                            total_generations=generaciones   
                        )
                        
                        # Evaluar
                        fitness_p_v, violc_p_v = evaluar_poblacion(pobl_prueba_validada, evaluate_fn)
                        
                        # Selección
                        poblacion, fitness_p, violc_p = seleccionar_mejores_deb(
                            poblacion, pobl_prueba_validada, fitness_p, violc_p, fitness_p_v, violc_p_v)
                        
                        # Mejor individuo de la generación (actualizar para la siguiente iteración)
                        x_best_for_current_gen, f_best_gen, v_best_gen = obtener_mejor_deb(poblacion, fitness_p, violc_p)
                        resultados.append({
                            'individuo': x_best_for_current_gen.copy(),
                            'fitness': f_best_gen,
                            'violation': v_best_gen,
                            'factible': v_best_gen <= 0
                        })
                        
                    tiempos.append(round(time.time() - inicio, 2))
                    matriz[:, j] = [r['fitness'] for r in resultados]
                    factibles_final.append(resultados[-1]['factible'])
                    violaciones_final.append(resultados[-1]['violation'])

                # --- Construimos el DataFrame ---
                cols = [f"Corrida {i+1}" for i in range(num_corridas)]
                df = pd.DataFrame(matriz, columns=cols)
                df.insert(0, "Generación", np.arange(1, generaciones + 1))

                # Calculamos promedio de fitness
                df["Promedio"] = df[cols].astype(float).mean(axis=1)

                # Fila de violaciones
                fila_viol = ["Violaciones"] + violaciones_final + [""]
                df.loc[len(df)] = fila_viol

                # Fila de tiempo
                fila_tiempo = ["Tiempo (s)"] + tiempos + [""]
                df.loc[len(df)] = fila_tiempo

                # Fila de factibles
                fila_fact = ["Factibles"] + [int(f) for f in factibles_final] + [""]
                df.loc[len(df)] = fila_fact

                # --- Guardamos en Excel ---
                sheet_name = nombre_lim[:31]  # Excel limita a 31 caracteres
                df.to_excel(writer, index=False, sheet_name=sheet_name)
                ws = writer.sheets[sheet_name]

                # Colorear celdas de factibles
                amarillo = PatternFill(start_color="FFFF00",
                                       end_color="FFFF00",
                                       fill_type="solid")
                fila_ultima_gen = generaciones + 1  # Excel es 1-based
                for col_idx, fact in enumerate(factibles_final, start=2):
                    if fact:
                        ws.cell(row=fila_ultima_gen, column=col_idx).fill = amarillo

            print(f"\n✅ Libro completo guardado: {archivo}")


In [14]:
offset = [1.91, 4.86, 3.74, 3.06, 0.79, 0.79, 0.29, 4.43, 3.07, 3.62]
#offset = [1.91, 4.86, 3.74, 3.06, 0.79, 0.79, 0.29, 4.43, 3.07, 3.62,0.58, 4.51, 2.13, 1.45, 3.23, 2.37, 0.69, 4.92, 1.98, 1.56,
#0.37, 4.77, 3.95, 1.11, 0.24, 2.78, 4.44, 2.65, 0.08, 5.10]

F               = 0.7    # factor de escala
CR              = 0.8    # tasa de cruza
tamano_poblacion= 100
generaciones = 2000
num_corridas    = 25


metodos_limite = [
    Limcor, 
    wrapper_evolutionary, 
    boundary,             
    reflection,          
    wrapping,            
    random_reset          
]

# Nombres descriptivos para los métodos 
nombres_limite = [
    "Limcor",
    "Evolutionary",
    "Boundary",
    "Reflection",
    "Wrapping",
    "Random Reset"
]

clases_problema = [ProblemaC01,ProblemaC02,ProblemaC03,ProblemaC04,ProblemaC05,ProblemaC06,
                   ProblemaC07,ProblemaC08,ProblemaC09,ProblemaC10,ProblemaC11,ProblemaC12,
                   ProblemaC13,ProblemaC14,ProblemaC15,ProblemaC16,ProblemaC17,ProblemaC18]

nombres_problemas = ["C01","C02","C03","C04","C05","C06","C07","C08","C09","C10","C11","C12","C13","C14","C15","C16","C17","C18"]


In [ ]:
print("iniciando proceso")

ejecutar_combinaciones_ED_limites(
    clases_problema,
    nombres_problemas,
    metodos_limite,
    nombres_limite,
    num_corridas,
    tamano_poblacion,
    offset,
    F,
    CR,
    metodo_mut,  
    generaciones
)

print("proceso finalizado")

iniciando proceso

🔁 Problema = C01 | Límite = Limcor

🔁 Problema = C01 | Límite = Evolutionary

🔁 Problema = C01 | Límite = Boundary

🔁 Problema = C01 | Límite = Reflection

🔁 Problema = C01 | Límite = Wrapping

🔁 Problema = C01 | Límite = Random Reset

✅ Libro completo guardado: ED_Limites_C01.xlsx

🔁 Problema = C02 | Límite = Limcor

🔁 Problema = C02 | Límite = Evolutionary

🔁 Problema = C02 | Límite = Boundary

🔁 Problema = C02 | Límite = Reflection

🔁 Problema = C02 | Límite = Wrapping

🔁 Problema = C02 | Límite = Random Reset

✅ Libro completo guardado: ED_Limites_C02.xlsx

🔁 Problema = C03 | Límite = Limcor

🔁 Problema = C03 | Límite = Evolutionary

🔁 Problema = C03 | Límite = Boundary

🔁 Problema = C03 | Límite = Reflection

🔁 Problema = C03 | Límite = Wrapping

🔁 Problema = C03 | Límite = Random Reset

✅ Libro completo guardado: ED_Limites_C03.xlsx

🔁 Problema = C05 | Límite = Limcor
